# Myers Brigss ML Project: Neural Net Model

### Imports & Dependencies

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os 
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from scipy.sparse import csr_matrix 

In [4]:
# import the cleaned df
cleaned_df = pd.read_csv('../../Resources/data/cleaned_mbti.csv', index_col=0)
cleaned_df.head()

,type,posts,http_count,no_url,text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,24,' and intj moments sportscenter not top ten...,and intj moments sportscenter not top ten p...
1,ENTP,'I'm finding the lack of me in these posts ver...,10,'I'm finding the lack of me in these posts ver...,i m finding the lack of me in these posts very...
2,INTP,'Good one _____ https://www.youtube.com/wat...,5,"'Good one _____ course, to which I say I k...",good one course to which i say i kn...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",2,"'Dear INTP, I enjoyed our conversation the o...",dear intp i enjoyed our conversation the ot...
4,ENTJ,'You're fired.|||That's another silly misconce...,6,'You're fired.|||That's another silly misconce...,you re fired that s another silly misconcepti...


### TFIDF Vectorizer

In [5]:
vectorizer = TfidfVectorizer(
    max_features=17000,
    min_df=7,
    max_df=0.8,
    stop_words="english",
    ngram_range=(1,3),
)
X = cleaned_df["text"].values
y = cleaned_df["type"].values
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
X_train = csr_matrix.toarray(X_train)
X_test = csr_matrix.toarray(X_test)

In [6]:
# Encode
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

### Model

In [7]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
# Define the model - deep neural net
number_input_features = 17000
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features,
                          kernel_constraint=max_norm(3), bias_constraint=max_norm(3), activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,
                             kernel_constraint=max_norm(3), bias_constraint=max_norm(3), activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=16, activation="softmax"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# Train the model
fit_model = nn.fit(X_train, y_train_categorical, callbacks = [es, mc], validation_split=0.33, epochs=100, batch_size=10)


Train on 6506 samples
Epoch 1/100
6506/6506 [==============================] - 7s 1ms/sample - loss: 2.3957 - accuracy: 0.2255
Epoch 2/100
6506/6506 [==============================] - 2s 270us/sample - loss: 1.9010 - accuracy: 0.4607
Epoch 3/100
6506/6506 [==============================] - 2s 250us/sample - loss: 1.3853 - accuracy: 0.6093
Epoch 4/100
6506/6506 [==============================] - 2s 312us/sample - loss: 0.9433 - accuracy: 0.7319
Epoch 5/100
6506/6506 [==============================] - 2s 274us/sample - loss: 0.6075 - accuracy: 0.8532
Epoch 6/100
6506/6506 [==============================] - 2s 241us/sample - loss: 0.3678 - accuracy: 0.9371
Epoch 7/100
6506/6506 [==============================] - 2s 242us/sample - loss: 0.2196 - accuracy: 0.9703
Epoch 8/100
6506/6506 [==============================] - 2s 241us/sample - loss: 0.1316 - accuracy: 0.9865
Epoch 9/100
6506/6506 [==============================] - 2s 241us/sample - loss: 0.0817 - accuracy: 0.9914
Epoch 10/100
6506

6506/6506 [==============================] - 2s 254us/sample - loss: 3.0712e-06 - accuracy: 1.0000
Epoch 76/100
6506/6506 [==============================] - 2s 250us/sample - loss: 2.7418e-06 - accuracy: 1.0000
Epoch 77/100
6506/6506 [==============================] - 2s 243us/sample - loss: 2.4548e-06 - accuracy: 1.0000
Epoch 78/100
6506/6506 [==============================] - 2s 241us/sample - loss: 2.2102e-06 - accuracy: 1.0000
Epoch 79/100
6506/6506 [==============================] - 2s 240us/sample - loss: 1.9792e-06 - accuracy: 1.0000
Epoch 80/100
6506/6506 [==============================] - 2s 238us/sample - loss: 1.7778e-06 - accuracy: 1.0000
Epoch 81/100
6506/6506 [==============================] - 2s 232us/sample - loss: 1.5953e-06 - accuracy: 1.0000
Epoch 82/100
6506/6506 [==============================] - 2s 232us/sample - loss: 1.4297e-06 - accuracy: 1.0000
Epoch 83/100
6506/6506 [==============================] - 2s 237us/sample - loss: 1.2837e-06 - accuracy: 1.0000
Epoch

In [ ]:
# Evaluate the model using the test data
train_loss, train_acc = nn.evaluate(X_train, y_train_categorical, verbose=0)
test_loss, test_acc = nn.evaluate(X_test, y_test_categorical, verbose=0)
print('accuracy:Train: %.3f, Test: %.3f' % (train_acc, test_acc))
print('loss:Train: %.3f, Test: %.3f' % (train_loss, test_loss))

In [ ]:
# summarize history for accuracy
print(fit_model.history.keys())

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
# summarize model accuracy
plt.plot(fit_model.history['accuracy'])
plt.plot(fit_model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(fit_model.history['loss'])
plt.plot(fit_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()